In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("train.csv")
df.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


In [3]:
updated = df.drop(['id', 'discourse_id' , 'discourse_start', 'discourse_end', 'discourse_type_num', 'predictionstring'], axis = 1)
updated.head()

,discourse_text,discourse_type
0,Modern humans today are always on their phone....,Lead
1,They are some really bad consequences when stu...,Position
2,Some certain areas in the United States ban ph...,Evidence
3,"When people have phones, they know about certa...",Evidence
4,Driving is one of the way how to get around. P...,Claim


In [4]:
updated['discourse_type'].unique()

array(['Lead', 'Position', 'Evidence', 'Claim', 'Concluding Statement',
       'Counterclaim', 'Rebuttal'], dtype=object)

#### Data Pre\-processing



In [5]:
evidence = updated[updated["discourse_type"] == "Evidence"]
evidence.dropna()
evidence = evidence.to_csv("evidence.csv", index = False)

In [6]:
claim = updated[updated["discourse_type"] == "Claim"]
claim.dropna()
claim = claim.to_csv("claim.csv", index = False)

In [7]:
conclusion = updated[updated["discourse_type"] == "Concluding Statement"]
conclusion.dropna()
conclusion = conclusion.to_csv("conclusion.csv", index = False)

In [8]:
counter = updated[updated["discourse_type"] == "Counterclaim"]
counter.dropna()
counter = counter.to_csv("counter.csv", index = False)

In [9]:
rebuttal = updated[updated["discourse_type"] == "Rebuttal"]
rebuttal.dropna()
rebuttal = rebuttal.to_csv("rebuttal.csv", index = False)

In [10]:
rebuttal = pd.read_csv("rebuttal.csv")
counter = pd.read_csv("counter.csv")
claim = pd.read_csv("claim.csv")
conclusion = pd.read_csv("conclusion.csv")
evidence = pd.read_csv("evidence.csv")

In [11]:
rebuttal = rebuttal.drop(['discourse_type'], axis=1)
rebuttal = rebuttal.to_csv("rebuttal1.csv", index = False)

In [12]:
counter = counter.drop(['discourse_type'],axis=1)
counter = counter.to_csv("counter1.csv", index = False)

In [13]:
claim = claim.drop(['discourse_type'],axis=1)
claim = claim.to_csv("claim1.csv", index = False)

In [14]:
conclusion = conclusion.drop(['discourse_type'],axis=1)
conclusion = conclusion.to_csv("conclusion1.csv", index = False)

In [15]:
evidence = evidence.drop(['discourse_type'],axis=1)
evidence = evidence.to_csv("evidence1.csv", index = False)

#### Creating Training Samples



In [16]:
# !pip3 install -q aitextgen

In [17]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import logging
logging.basicConfig(
        format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO
    )
from sklearn.model_selection import train_test_split
from aitextgen import aitextgen
from aitextgen.TokenDataset import TokenDataset
from aitextgen.utils import build_gpt2_config
from aitextgen.tokenizers import train_tokenizer
import pandas as pd
import numpy as np

In [19]:
rebuttal1 = pd.read_csv("rebuttal1.csv")
counter1 = pd.read_csv("counter1.csv")
claim1 = pd.read_csv("claim1.csv")
conclusion1 = pd.read_csv("conclusion1.csv")
evidence1 = pd.read_csv("evidence1.csv")

In [24]:
evidence1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45702 entries, 0 to 45701
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   discourse_text  45702 non-null  object
dtypes: object(1)
memory usage: 357.2+ KB


In [19]:
train_r, testr = train_test_split(rebuttal1, test_size=0.25)
train_cou, testcou = train_test_split(counter1, test_size=0.25)
train_cl, testcl = train_test_split(claim1, test_size=0.25)
train_con, testcon = train_test_split(conclusion1, test_size=0.25)
train_e, teste = train_test_split(evidence1, test_size=0.25)

In [20]:
training_samples_r = train_r.discourse_text.values.tolist()
training_samples_cou = train_cou.discourse_text.values.tolist()
training_samples_cl = train_cl.discourse_text.values.tolist()
training_samples_con = train_con.discourse_text.values.tolist()
training_samples_e = train_e.discourse_text.values.tolist()

#### Creating TokenDatasets



In [29]:
training_dataset_r = TokenDataset(texts=training_samples_r, save_cache=True)
training_file_r = "rebuttal/dataset_cache.tar.gz"

07/07/2022 18:38:32 — INFO — aitextgen.TokenDataset — Encoding 3,252 texts.


  0%|          | 0/3252 [00:00<?, ?it/s]

07/07/2022 18:38:33 — INFO — aitextgen.TokenDataset — Caching and compressing dataset to dataset_cache.tar.gz


In [22]:
training_dataset_cou = TokenDataset(texts=training_samples_cou, save_cache=True)
training_file_cou = "counter/dataset_cache.tar.gz"

07/07/2022 18:37:55 — INFO — aitextgen.TokenDataset — Encoding 4,362 texts.


  0%|          | 0/4362 [00:00<?, ?it/s]

07/07/2022 18:37:55 — INFO — aitextgen.TokenDataset — Caching and compressing dataset to dataset_cache.tar.gz


In [23]:
training_dataset_cl = TokenDataset(texts=training_samples_cl, save_cache=True)
training_file_cl = "claim/dataset_cache.tar.gz"

07/07/2022 18:37:56 — INFO — aitextgen.TokenDataset — Encoding 37,656 texts.


  0%|          | 0/37656 [00:00<?, ?it/s]

07/07/2022 18:37:57 — INFO — aitextgen.TokenDataset — Caching and compressing dataset to dataset_cache.tar.gz


In [24]:
training_dataset_con = TokenDataset(texts=training_samples_con, save_cache=True)
training_file_con = "conclusion/dataset_cache.tar.gz"

07/07/2022 18:37:57 — INFO — aitextgen.TokenDataset — Encoding 10,128 texts.


  0%|          | 0/10128 [00:00<?, ?it/s]

07/07/2022 18:37:58 — INFO — aitextgen.TokenDataset — Caching and compressing dataset to dataset_cache.tar.gz


In [25]:
training_dataset_e = TokenDataset(texts=training_samples_e, save_cache=True)
training_file_e = "evidence/dataset_cache.tar.gz"

07/07/2022 18:37:58 — INFO — aitextgen.TokenDataset — Encoding 34,276 texts.


  0%|          | 0/34276 [00:00<?, ?it/s]

07/07/2022 18:38:02 — INFO — aitextgen.TokenDataset — Caching and compressing dataset to dataset_cache.tar.gz


#### Loading the models



In [26]:
ai_tuned = aitextgen(model="EleutherAI/gpt-neo-125M", to_gpu=True)

07/07/2022 18:38:03 — INFO — aitextgen — Loading EleutherAI/gpt-neo-125M model from /aitextgen.


07/07/2022 18:38:05 — INFO — aitextgen — Using the tokenizer for EleutherAI/gpt-neo-125M.


loading file https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/vocab.json from cache at aitextgen/08c00c4159e921d4c941ac75732643373aba509d9b352a82bbbb043a94058d98.a552555fdda56a1c7c9a285bccfd44ac8e4b9e26c8c9b307831b3ea3ac782b45


loading file https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/merges.txt from cache at aitextgen/12305762709d884a770efe7b0c68a7f4bc918da44e956058d43da0d12f7bea20.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b


loading file https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/tokenizer.json from cache at None


loading file https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/added_tokens.json from cache at None


loading file https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/special_tokens_map.json from cache at aitextgen/6c3239a63aaf46ec7625b38abfe41fc2ce0b25f90800aefe6526256340d4ab6d.2b8bf81243d08385c806171bc7ced6d2a0dcc7f896ca637f4e777418f7f0cc3c


loading file https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/tokenizer_config.json from cache at aitextgen/3cc88b3aa29bb2546db2dc21783292e2a086bb7158c7b5ceddeb24158a85c183.e74f7c3643ee79eb023ead36008be72fe726dada60fa3b2a0569925cfefa1e74


07/07/2022 18:38:08 — INFO — aitextgen — GPTNeo loaded with 125M parameters.


#### Initialising Training



In [91]:
ai_tuned.train(training_file_e,
         line_by_line=False,
         from_cache=True,
         num_steps=3000,
         generate_every=500,
         save_every=500,
         learning_rate=1e-3,
         batch_size=1
         output_dir = "evidence/")

07/05/2022 20:19:40 — INFO — aitextgen — Loading text from evidence/dataset_cache.tar.gz with generation length of 2048.


07/05/2022 20:19:40 — INFO — aitextgen.TokenDataset — TokenDataset containing 3,103,713 subsets loaded via cache.


07/05/2022 20:19:40 — INFO — torch.distributed.nn.jit.instantiator — Created a temporary directory at /tmp/tmp4bg401hc


07/05/2022 20:19:40 — INFO — torch.distributed.nn.jit.instantiator — Writing /tmp/tmp4bg401hc/_remote_module_non_scriptable.py


/projects/a14fcf5c-51ae-40cd-99cb-a5783a82837c/miniconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:151: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  rank_zero_deprecation(
/projects/a14fcf5c-51ae-40cd-99cb-a5783a82837c/miniconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:96: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
/projects/a14fcf5c-51ae-40cd-99cb-a5783a82837c/miniconda3/lib/python3.9/site-packages/pytorch_lightning/trainer

...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
3,000 steps reached: saving model to /evidence/


3,000 steps reached: generating sample texts.



	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


07/05/2022 21:03:30 — INFO — aitextgen — Saving trained model pytorch_model.bin to /evidence/


In [30]:
ai_tuned.train(training_file_r,
         line_by_line=True,
         from_cache=True,
         num_steps=4000,
         generate_every=500,
         save_every=500,
         learning_rate=1e-3,
         batch_size=1,
         output_dir = "rebuttal/")

07/07/2022 18:38:37 — INFO — aitextgen — Loading text from rebuttal/dataset_cache.tar.gz with generation length of 2048.


07/07/2022 18:38:37 — INFO — aitextgen.TokenDataset — TokenDataset containing 106,948 subsets loaded via cache.


07/07/2022 18:38:38 — INFO — torch.distributed.nn.jit.instantiator — Created a temporary directory at /tmp/tmpoqmtcmz0


07/07/2022 18:38:38 — INFO — torch.distributed.nn.jit.instantiator — Writing /tmp/tmpoqmtcmz0/_remote_module_non_scriptable.py


07/07/2022 18:38:38 — WARNING — aitextgen — pytorch_model.bin already exists in /rebuttal/ and will be overwritten!


...

4,000 steps reached: generating sample texts.


However, it is still possible to get the same amount of learning and ratio of learning not only at school, which will make you feel learning more over the summer.


	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


07/07/2022 19:44:56 — INFO — aitextgen — Saving trained model pytorch_model.bin to /rebuttal/


In [31]:
ai_tuned.train(training_file_cl,
         line_by_line=True,
         from_cache=True,
         num_steps=4000,
         generate_every=500,
         save_every=500,
         learning_rate=1e-3,
         batch_size=1,
         output_dir = "claim/")

07/07/2022 19:45:00 — INFO — aitextgen — Loading text from claim/dataset_cache.tar.gz with generation length of 2048.


07/07/2022 19:45:00 — INFO — aitextgen.TokenDataset — TokenDataset containing 792,736 subsets loaded via cache.


07/07/2022 19:45:00 — WARNING — aitextgen — pytorch_model.bin already exists in /claim/ and will be overwritten!


07/07/2022 19:45:00 — INFO — pytorch_lightning.utilities.rank_zero — GPU available: True, used: True


07/07/2022 19:45:00 — INFO — pytorch_lightning.utilities.rank_zero — TPU available: False, using: 0 TPU cores


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
4,000 steps reached: saving model to /claim/


4,000 steps reached: generating sample texts.


 even our bodies once have been shown that the Face was not created by allien lifeforms 


	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


07/07/2022 20:51:40 — INFO — aitextgen — Saving trained model pytorch_model.bin to /claim/


In [32]:
ai_tuned.train(training_file_cou,
         line_by_line=True,
         from_cache=True,
         num_steps=4000,
         generate_every=500,
         save_every=500,
         learning_rate=1e-3,
         batch_size=1,
         output_dir = "counter/")

07/07/2022 20:51:44 — INFO — aitextgen — Loading text from counter/dataset_cache.tar.gz with generation length of 2048.


07/07/2022 20:51:44 — INFO — aitextgen.TokenDataset — TokenDataset containing 122,620 subsets loaded via cache.


07/07/2022 20:51:44 — WARNING — aitextgen — pytorch_model.bin already exists in /counter/ and will be overwritten!


07/07/2022 20:51:44 — INFO — pytorch_lightning.utilities.rank_zero — GPU available: True, used: True


07/07/2022 20:51:44 — INFO — pytorch_lightning.utilities.rank_zero — TPU available: False, using: 0 TPU cores


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
3,500 steps reached: saving model to /counter/


3,500 steps reached: generating sample texts.


Although the electoral college does wrong, irrational and even irrational things that do matter have it's sometimes suer of voting and for different reasons. 


In [0]:
ai_tuned.train(training_file_con,
         line_by_line=True,
         from_cache=True,
         num_steps=4000,
         generate_every=500,
         save_every=500,
         learning_rate=1e-3,
         batch_size=1,
         output_dir = "conclusion/")

#### Generating Data



In [20]:
evi = aitextgen(model_folder="evidence/", to_gpu=True)
con = aitextgen(model_folder="conclusion/", to_gpu=True)
cl = aitextgen(model_folder="claim/", to_gpu=True)
reb = aitextgen(model_folder="rebuttal/", to_gpu=True)
cou = aitextgen(model_folder="counter/", to_gpu=True)

07/07/2022 18:07:37 — INFO — aitextgen — Loading model from provided weights and config in /evidence/.


07/07/2022 18:07:42 — INFO — aitextgen — GPTNeo loaded with 125M parameters.


07/07/2022 18:07:42 — INFO — aitextgen — Using the default GPT-2 Tokenizer.


07/07/2022 18:07:46 — INFO — aitextgen — Loading model from provided weights and config in /conclusion/.


07/07/2022 18:07:51 — INFO — aitextgen — GPTNeo loaded with 125M parameters.


...

07/07/2022 18:08:01 — INFO — aitextgen — GPTNeo loaded with 125M parameters.


07/07/2022 18:08:01 — INFO — aitextgen — Using the default GPT-2 Tokenizer.


07/07/2022 18:08:01 — INFO — aitextgen — Loading model from provided weights and config in /counter/.


07/07/2022 18:08:06 — INFO — aitextgen — GPTNeo loaded with 125M parameters.


07/07/2022 18:08:06 — INFO — aitextgen — Using the default GPT-2 Tokenizer.


In [21]:
evi.generate(n=3, batch_size=1, prompt="I wanna dance with somebody", max_length=1000, temperature=1, top_p=0.9)

I wanna dance with somebody. The more people you ask people with the same mind. We are all different, everyone has a different mind. Asking a bunch of people with many minds can be nerve racking and they are all different. 
I wanna dance with somebody or be the best to ask about if the principal should let the students to use there phone. 
I wanna dance with somebody and ask them to join, and maybe even with someone with cancer and a disability they might not feel well, and what about other people who may not like it. 


In [22]:
con.generate(n=3, batch_size=1, prompt="I wanna dance with somebody", max_length=1000, temperature=1, top_p=0.9)

I wanna dance with somebody who has that heart on them making happy. So if you want someone to have that heart then join the best for you.
I wanna dance with somebody that has one face on it. Once I finish this I learned that it was just another face-according to a landform. 
I wanna dance with somebody that wants the advice for me. When I am speaking for the advice, I do not see how it will affect the other peoples views. But the other advice we have not came to head. Let's just be able to keep it that way and not have to follow someone's dreams.  


In [23]:
cou.generate(n=3, batch_size=1, prompt="I wanna dance with somebody", max_length=1000, temperature=1, top_p=0.9)

I wanna dance with somebody else's an example of voting for president my county voted for a slate of electors who will choose the president. 
I wanna dance with somebody elses grades, it could be a little hard.
I wanna dance with somebody elses grades at a young age and it may be alot else 


In [24]:
cl.generate(n=3, batch_size=1, prompt="I wanna dance with somebody", max_length=1000, temperature=1, top_p=0.9)

I wanna dance with somebody thats best friend
I wanna dance with somebody thats giving me my opinion on someone thats giving me the right information, I cant just think it helps me get a feeling on something or something.
I wanna dance with somebody thats giving me there opinion on someone because i dont like any one i hate when someone gives you there opinion about something


In [25]:
reb.generate(n=3, batch_size=1, prompt="I wanna dance with somebody", max_length=1000, temperature=1, top_p=0.9)

I wanna dance with somebody and if you remember what you are doing and are willing to stay quiet incase something happens. 
I wanna dance with somebody and if you remember what you are doing you get to be careful with making decisions, I think every child is well being dropped off is not going to take that privilege away from them and giving them to complete the projects seriously. Also, if the students come up with an idea and chose what they want to do, the project would be no educational, it would also cause lack of other students trying to learn anything. So, the lack of student-designed projects would prevent them from being lazy when it starts.
I wanna dance with somebody who doesn't see a problem with this idea and it doesn't help with many things. Like just a few, or maybe they can't find a little bit of work to do. If the students wants a teacher that does not know what they need to know, they can ask for help from a teacher, friend, or a friend peers. 


In [45]:
"distilgpt2"

3

In [45]:
cl.generate(n=3, batch_size=1, prompt="Dogs are the best", max_length=1000, temperature =0.9, top_p=0.9)

Dogs are the best thing to do in a country.
Dogs are the best thing to ever encounter
Dogs are the best way to get from getting bad grades, they'd make it to where they'll only be able to participate in school activities if they're willing to. 


In [46]:
evi.generate(n=3, batch_size=1, prompt="A woman is murdered", max_length=500, temperature =0.9, top_p=0.9)

A woman is murdered at the time when she is sad. She is angry and is angry at that moment. It is not a right thing to do, to do anything, to get her into a fight. You have to do what you want to do with her/he, and to make her/he feel free.
A woman is murdered in her house, and is not being able to do anything. There are many different things that go wrong in this world, so that the person that was in the situation that was in the situation that was in the first place is able to prove what they do. If the person that went through that event was someone who had gone through a similar thing and has the same type of life, then they are able to prove what they have to say. 
A woman is murdered because she was not her best friend and wanted to get revenge for her. But now she is not really that bad and she has nothing to do with her life. Now she has been through more and more to the point. She is sad and not happy to her. But she is truly happy and not to her. 
